### **Python Data Engineering Project**
##### *-- Anh Vi Pham --*

### **Setup Database**

#### Connect Server

In [ ]:
import pyodbc
import json
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

# database credentials
server = 'localhost,1433' 
database = 'master' 
driver = '{ODBC Driver 18 for SQL Server}'

# import keys from json file
with open("database_keys.json", "r") as file:
    config = json.load(file)
username = config["username"]
password = config["password"]

# establish connecttion

conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes')
cursor = conn.cursor()

#### Create new database

In [2]:
def get_current_database(cursor=cursor):
    cursor.execute("SELECT DB_NAME() AS CurrentDatabase")
    current_db = cursor.fetchone()[0]
    return print(f"Currently connected to database: {current_db}")
get_current_database(cursor)

Currently connected to database: master


In [3]:
# drop database
conn.autocommit = True

# Drop the database if it exists
cursor.execute("IF EXISTS (SELECT * FROM sys.databases WHERE name = 'HospitalOperation') DROP DATABASE HospitalOperation")
print("Database 'HospitalOperation' dropped successfully if it existed.")

Database 'HospitalOperation' dropped successfully if it existed.


In [4]:
conn.autocommit = True
try:
    cursor.execute("CREATE DATABASE HospitalOperation")
    print("Database 'HospitalOperation' created successfully.")
except Exception as e:
    print(f"Error: {e}")

conn.autocommit = False

Database 'HospitalOperation' created successfully.


In [ ]:
conn.close()  # Close the current connection to the master database

#### Connect to new database

In [16]:
new_database_name = 'HospitalOperation'
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={new_database_name};UID={username};PWD={password};TrustServerCertificate=yes')
cursor = conn.cursor()

In [17]:
get_current_database(cursor)

Currently connected to database: HospitalOperation


#### Load database configuration file from table_creation_code.txt

In [7]:
def load_sql_from_txt(file_name: str, mode='r'):
    commands = []
    with open(file_name, mode) as file:
        imported_commands = file.read().split(';')
        for com in imported_commands:
            commands.append(com.replace('\n', ' ').strip())
    return commands

In [8]:
try:
    db_file = 'table_creation_code.txt'
    table_creation_codes = load_sql_from_txt(db_file)
    for command in table_creation_codes:
        if command: 
            cursor.execute(command)

    conn.commit()
    print('Database configurated succesfull from')

except Exception as e:
    print('Database configurated failed')
    print(e)


Database configurated succesfull from


#### Load data from sample_dataset.txt

In [9]:
try:
    data_file = 'sample_dataset.txt'
    data_codes = load_sql_from_txt(data_file)
    for command in data_codes:
        if command: 
            cursor.execute(command)

    conn.commit()
    print('Data loaded succesfully')

except Exception as e:
    print('Data loaded failed')
    print(e)


Data loaded succesfully


### **Query**

In [18]:
def query(sql_code, engine = conn):
    df_query = pd.read_sql(sql_code, engine)
    return df_query

In [19]:
df_patient = query('''
                   SELECT * FROM Patient
                   ''')

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12080\165815039.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_query = pd.read_sql(sql_code, engine)


### **Close Connection**

In [ ]:
# conn.close()